# Install Library

In [ ]:
%pip install autogen-agentchat~=0.2

# Setup

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")

llm_config = [{"model": azure_deployment,
               "api_type": "azure",
               "temperature": 0.9,
               "api_key": api_key,
               "base_url": azure_endpoint,
               "api_version": "2024-02-15-preview"}] 

# Define an AutoGen ConversableAgent

In [ ]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

In [ ]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke about Autogen.", "role": "user"}]
)
print(reply)

# Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [ ]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=3,
)

# Print out results:
### a. Chat history
### b. Cost
### c. Summary of the conversation

In [ ]:
import pprint
pprint.pprint(chat_result.chat_history)

In [ ]:
pprint.pprint(chat_result.cost)

In [ ]:
pprint.pprint(chat_result.summary)

# Get a better summary of the conversation

In [ ]:
#Parameters in initiate_chat
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling. Give me a joke about Autogen.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

In [ ]:
pprint.pprint(chat_result.summary)

# How to Control Termination with AutoGen? 
## Currently there are two broad mechanism to control the termination of conversations between agents:

### a. Specify parameters in initiate_chat: When initiating a chat, you can define parameters that determine when the conversation should end.
### b. Configure an agent to trigger termination: When defining individual agents, you can specify parameters that allow agents to terminate of a conversation based on particular (configurable) conditions. Currently, there are two parameters you can configure:
#### b.1.  max_consecutive_auto_reply: This condition triggers termination if the number of automatic responses to the same sender exceeds a threshold. You can customize this using the max_consecutive_auto_reply argument of the ConversableAgent class. To accomplish this the agent maintains a counter of the number of consecutive automatic responses to the same sender. Note that this counter can be reset because of human intervention. We will describe this in more detail in the next chapter.
#### b.2. is_termination_msg: This condition can trigger termination if the received message satisfies a particular condition, e.g., it contains the word “TERMINATE”. You can customize this condition using the is_terminate_msg argument in the constructor of the ConversableAgent class.

In [ ]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. ",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

#max_consecutive_auto_reply
joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. ",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
    max_consecutive_auto_reply=2,
)

chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling. We end the conversation when we feel like it.",
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

In [ ]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. ",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

# is_termination_msg sample
joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go' or 'Goodbye'.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling. We end the conversation when we feel like it.",
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

In [ ]:
pprint.pprint(chat_result.summary)

# Human Input Modes
## The mode is specified through the human_input_mode argument of the ConversableAgent. The modes are:
### 1. NEVER: human input is never requested.
### 2. TERMINATE (default): human input is only requested when a termination condition is met. Note that in this mode if the human chooses to intercept and reply, the conversation continues and the counter used by max_consecutive_auto_reply is reset.
### 3. Human input is always requested and the human can choose to skip and trigger an auto-reply, intercept and provide feedback, or terminate the conversation. Note that in this mode termination based on max_consecutive_auto_reply is ignored.

In [ ]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. ",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

# Human input mode TERMINATE
joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. ",
    llm_config={"config_list": llm_config},
    human_input_mode="TERMINATE",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling. We end the conversation when we feel like it.",
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling. We end the conversation when we feel like it.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Great to have you here. So, I went to the doctor the other day. He told me I had to stop playing the guitar. And I was like, "Why, doc?" He said, "Because you're terrible at it!" 

Your turn, Joe. Why don't you hit me with a joke?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
joe (to cathy):

Hey Cathy! That was a good one. Alright, here’s mine:

Why don’t scientists trust atoms anymore?

Because they make up everything!

--------------------------------------------------------------------------------
cathy (to joe):

Nice one, Joe! I love a good science joke. Alright, here’s another one for you:

Why did the scarecrow win an award?

Because he was outstanding in his field!

---------------------------------

/workspaces/generative-ai-for-beginners/.venv/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:1461: UserWarning: Extracted_response from ChatCompletion(id='chatcmpl-ASDhPsqBOMUQjv2FSnP2hibe8a6u5', choices=[Choice(finish_reason='content_filter', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': True, 'severity': 'medium'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1731289207, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_67802d9a6d', usage=CompletionUsage(completion_tokens=38, prompt_tokens=742, total_tokens=780), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {}}], cost=0.00428, message_retrieval_function=<bound method OpenAIClient.message_r

joe (to cathy):

Cathy, you've been awesome! But I gotta go. Catch you next time! Goodbye!

--------------------------------------------------------------------------------
cathy (to joe):

You've been fantastic too, Joe! Thanks for all the laughs. Catch you next time! Goodbye!

--------------------------------------------------------------------------------


In [18]:
pprint.pprint(chat_result.summary)

('Joe and Cathy exchanged a series of jokes, starting with general jokes and '
 "then focusing on Autogen-themed humor. They enjoyed each other's company, "
 'shared laughs, and parted on good terms, expressing appreciation for the fun '
 'interaction.')


In [19]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. ",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

# Human input mode ALWAYS
joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. ",
    llm_config={"config_list": llm_config},
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling. We end the conversation when we feel like it.",
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling. We end the conversation when we feel like it.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Great to have you here. So, I went to the doctor the other day. He told me I had to stop playing the guitar. And I was like, "Why, doc?" He said, "Because you're terrible at it!" 

Your turn, Joe. Why don't you hit me with a joke?

--------------------------------------------------------------------------------
joe (to cathy):

Make a joke regarding Autogen

--------------------------------------------------------------------------------
cathy (to joe):

Sure thing, Joe! So, you know Autogen, right? That software that writes text automatically? I heard it tried to write a love letter the other day...

It started with, "Dear [Insert Beloved's Name],"

Turns out, even AI needs some help in the romance department!

-------------------------------------------------------------